In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('freight_data/FAF5.6.1_State_2018-2023.csv')
meta_df = pd.read_excel('freight_data/FAF5_metadata.xlsx', sheet_name=None)

### Read metadata

In [3]:
metadata_filepath = 'freight_data/FAF5_metadata.xlsx'
for sheet_name, _ in meta_df.items():
    print(f"\nSheet: {sheet_name}")

In [4]:
state_labels = pd.read_excel(metadata_filepath, sheet_name='State')
trade_type = pd.read_excel(metadata_filepath, sheet_name='Trade Type')
commodity = pd.read_excel(metadata_filepath, sheet_name='Commodity (SCTG2)')

In [5]:
df.columns

## Goal: Analyze domestic shipments in and out of Georgia.

### According to metadata, the state of Georgia has numeric label 13 and all domestic flow is labelled 1 in trade type. Dropping foreign flow columns

In [6]:
domestic_df = df[((df.dms_destst == 13) | (df.dms_origst == 13)) & df.trade_type == 1]

foreign_columns = [col for col in domestic_df.columns if 'fr_' in col] + ['trade_type']
domestic_df = domestic_df.drop(columns = foreign_columns)

In [7]:
domestic_df

#### Day 2
### Selecting customer oriented commodities

Meat/seafood, Alcoholic beverages, Tobacco prods., Pharmaceuticals (5, 8, 9, 21) - potentially customer oriented commodities

In [8]:
commodity_mapping = {'5': 'Meat/seafood', '8': 'Alcoholic beverages', '9': 'Tobacco prods.', '21': 'Pharmaceuticals'}
customer_specific_df = domestic_df[(domestic_df.sctg2 == 5) | (domestic_df.sctg2 == 8) | (domestic_df.sctg2 == 9) | (domestic_df.sctg2 == 21)]

customer_specific_df

### Read and join Population, median income and 

In [52]:
state_pop19 = pd.read_excel('freight_data/nst-est2019-01.xlsx', header=3, index_col=0)
state_pop24 = pd.read_excel('freight_data/NST-EST2024-POP.xlsx', header=3, index_col=0)

# state_pop00 = pd.read_csv('freight_data/st-est00int-agesex.csv') #header=3, index_col=0)

In [53]:
# state_pop00 = state_pop00[state_pop00.NAME == 'Georgia']
# # print(state_pop00.columns)
# state_pop00 = state_pop00[['AGE','ESTIMATESBASE2000','POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009']]
# state_pop00.columns = ['AGE','2000','2007', '2008', '2009']
# 
# # state_pop00 = pd.DataFrame(state_pop00.sum(), columns=['.Georgia'])
# # state_pop00.columns = ['.Georgia']
# print(state_pop00.columns)
# print(state_pop00.sum())
# state_pop00

In [11]:
georgia_state_pop = state_pop19.loc['.Georgia']
georgia_state_pop = pd.concat([georgia_state_pop, state_pop24.loc['.Georgia']])
georgia_state_pop = georgia_state_pop.drop('Unnamed: 1')
georgia_state_pop.loc[2010:2024]

### Median Income and Retail Trade Georgia
- MEHOINUSGAA646N - Median Household Income with value of USD for that particular year.
- MEHOINUSGAA672N - Real Median Household Income accounting for inflation.
- GARETAILNQGSP - Georgia Retail Trade GDP 

In [12]:
retails = pd.read_csv('freight_data/fredgraph.csv')
retails['observation_date'] = pd.to_datetime(retails['observation_date'])
retails['year'] = retails['observation_date'].dt.year
retails = retails.drop(columns=['observation_date'])
retails = retails.set_index('year')

In [13]:
retails

In [14]:
additional_info_df = pd.concat([retails, georgia_state_pop], axis=1)
additional_info_df = additional_info_df.dropna()
additional_info_df = additional_info_df.rename(columns={".Georgia": "Population"})
additional_info_df

### Aggregate shipment data based on both commodity and inbound/outbound

In [15]:
# aggregated_shipment = customer_specific_df.groupby(by=['sctg2','dms_destst']).agg('sum')
inbound_shipments = customer_specific_df[customer_specific_df.dms_destst == 13]
outbound_shipments = customer_specific_df[customer_specific_df.dms_origst == 13]
inbound_shipments = inbound_shipments.drop(columns=['dms_destst'])
outbound_shipments = outbound_shipments.drop(columns=['dms_origst'])

aggregated_inbound = inbound_shipments.groupby(by=['sctg2']).agg('sum')
aggregated_outbound = outbound_shipments.groupby(by=['sctg2']).agg('sum')

aggregated_inbound

In [16]:
tons_and_value_columns = [col_name for col_name in aggregated_inbound.columns if 'tons_' in col_name or 'value_' in col_name]
annual_shipments_in = aggregated_inbound[tons_and_value_columns]
annual_shipments_out = aggregated_outbound[tons_and_value_columns]
annual_shipments_in = annual_shipments_in.transpose()
annual_shipments_in

Dataframe transformation
- Extract metric and year separately and store in separate columns

In [17]:
def separate_metric_and_year(dataframe, metric_substring):
    """
    Assumes the metric to be in the format 'metric_name_{year}'.
    """
    metric_df = dataframe[dataframe.index.str.startswith(metric_substring)]
    metric_df.columns = [metric_substring+str(col_name) for col_name in metric_df.columns]
    metric_df.index = metric_df.index.str.replace(metric_substring, '')
    return metric_df

transformed_df = annual_shipments_in.copy()
value_df = separate_metric_and_year(transformed_df, 'value_')
tons_df = separate_metric_and_year(transformed_df, 'tons_')
cvalue_df = separate_metric_and_year(transformed_df, 'current_value_')

Make both index years same datetime format


In [18]:
annual_shipments_inbound = pd.concat([tons_df, value_df, cvalue_df], axis=1)
annual_shipments_inbound['year'] = pd.to_datetime(annual_shipments_inbound.index)
annual_shipments_inbound['year'] = annual_shipments_inbound['year'].dt.year
annual_shipments_inbound = annual_shipments_inbound.set_index('year')
annual_shipments_inbound = pd.concat([annual_shipments_inbound, additional_info_df], axis=1)
annual_shipments_inbound = annual_shipments_inbound.dropna()
# MEHOINUSGAA646N does not account for inflation.
annual_shipments_inbound = annual_shipments_inbound.drop(columns=['MEHOINUSGAA646N'])


In [19]:
annual_shipments_inbound

#### Day 3
Verifying units and standardize.
- tons - (thousand tons)
- value and current_value - (million dollars)
- Median Income - (dollars)
- Retail Trade GDP - (million dollars)
- Population - (-)

In [20]:
annual_shipments_inbound['MEHOINUSGAA672N'] = annual_shipments_inbound['MEHOINUSGAA672N'].apply(lambda x: x*10e-6)
annual_shipments_inbound

In [21]:
scaler = StandardScaler()
shipment_features = annual_shipments_inbound[['MEHOINUSGAA672N','GARETAILNQGSP', 'Population']]
scaled_shipments = scaler.fit_transform(shipment_features)
scaled_shipments = pd.DataFrame(scaled_shipments, index=shipment_features.index, columns=shipment_features.columns)

scaled_shipments

In [22]:
annual_shipments_inbound[['MEHOINUSGAA672N','GARETAILNQGSP','Population']] = shipment_features[['MEHOINUSGAA672N','GARETAILNQGSP','Population']]
# annual_shipments_inbound['GARETAILNQGSP'] = shipment_features['GARETAILNQGSP']
# annual_shipments_inbound['Population'] = shipment_features['Population']

In [23]:
annual_shipments_inbound

### Correlation Analysis

In [24]:
main_features = ['MEHOINUSGAA672N','GARETAILNQGSP','Population']
selected_features = annual_shipments_inbound[['tons_5']+main_features]

selected_features.corr()

In [25]:
feature_correlations = annual_shipments_inbound.corr()

feature_correlations = feature_correlations.drop(columns=['tons_5', 'tons_8', 'tons_9', 'tons_21', 'value_5', 'value_8','value_9', 'value_21', 'current_value_5', 'current_value_8','current_value_9', 'current_value_21'])

In [26]:
feature_correlations

#### Day 7

In [24]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import plotly.express as px

In [25]:
cleaned_df = pd.read_csv('freight_data/Georgia_Annual_Inbound_Shipments_2012-2023.csv', index_col=0)

In [26]:
cleaned_df

In [27]:
scaler = StandardScaler()
shipment_features = cleaned_df[['MEHOINUSGAA672N','GARETAILNQGSP', 'Population']]
scaled_shipments = scaler.fit_transform(shipment_features)
scaled_shipments = pd.DataFrame(scaled_shipments, index=shipment_features.index, columns=shipment_features.columns)

scaled_shipments

In [28]:
cleaned_df[['MEHOINUSGAA672N','GARETAILNQGSP','Population']] = shipment_features[['MEHOINUSGAA672N','GARETAILNQGSP','Population']]
feature_correlations = cleaned_df.corr()

feature_correlations = feature_correlations.drop(columns=['tons_5', 'tons_8', 'tons_9', 'tons_21', 'value_5', 'value_8','value_9', 'value_21'])

In [29]:
feature_correlations

#### Predict the commodities that will grow fastest

In [30]:
cleaned_df[['MEHOINUSGAA672N','GARETAILNQGSP','Population']] = scaled_shipments[['MEHOINUSGAA672N','GARETAILNQGSP','Population']]

In [31]:
cleaned_df

Add lagged value for commodities to be used as feature for future prediction: value_commodity_t-1

In [32]:
def add_lagging_commodity(data_frame):
    tons_commodity = [col for col in data_frame.columns if 'tons' in col]
    value_commodity = [col for col in data_frame.columns if 'value' in col]
    for col in tons_commodity:
        data_frame[f'{col}_lagged'] = data_frame[col].shift(1)
    for col in value_commodity:
        data_frame[f'{col}_lagged'] = data_frame[col].shift(1)
    
    return data_frame

In [34]:
updated_cleaned_df = add_lagging_commodity(cleaned_df)
updated_cleaned_df = updated_cleaned_df.dropna()
updated_cleaned_df

In [35]:
tons_columns = ['tons_5', 'tons_8', 'tons_9', 'tons_21']
value_columns = ['value_5', 'value_8', 'value_9', 'value_21']
fig = px.line(updated_cleaned_df, y= tons_columns, 
              title='Quantity of commodity shipped domestically in Georgia (2013-2023)')
fig.update_layout(xaxis_title='Year', yaxis_title='Quantity (Million Tons)')
fig.show()

In [37]:
fig = px.line(updated_cleaned_df, y= value_columns, 
              title='Value of commodity shipped domestically in Georgia (2013-2023)')
fig.update_layout(xaxis_title='Year', yaxis_title='Value (Million Dollars)')
fig.show()

Dropping 2023 for final forecast due to lack of 2024 data for features

In [39]:
final_forecast_data = updated_cleaned_df
updated_cleaned_df = updated_cleaned_df.drop(2023)
updated_cleaned_df

In [40]:
forecast_df = pd.read_csv('freight_data/forecast_prep.csv', index_col=0)

## Day 10

Check differences between FAF4 and 5 data as well as preprocessing data differences

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils.visualization import filter_georgia_value, commodity_based_value_trend, plot_from_data_dict

In [31]:
df_2012 = pd.read_csv('freight_data/FAF5.6.1_Reprocessed_1997-2012_State.csv')
df_2012faf4 = pd.read_csv('freight_data/FAF4.5.1_State.csv')
df_2013 = pd.read_csv('freight_data/FAF4.5.1_State_2013.csv')
df_2016 = pd.read_csv('freight_data/FAF4.5.1_State_2016.csv')
df_2017faf4 = pd.read_csv('freight_data/FAF4.5.1_State_2017.csv')
df_2017faf5 = pd.read_csv('freight_data/FAF5.6.1_State.csv')
df_2018faf4 = pd.read_csv('freight_data/FAF4.5.1_State_2018.csv')
df_2018faf5 = pd.read_csv('freight_data/FAF5.6.1_State_2018-2023.csv')

In [64]:
# def filter_georgia_value(data_frame, year, commodity_code):
#     relevant_cols = [col for col in data_frame.columns if 'dms_dest' in col or 'sctg2' in col]
#     year_specific_col = [col for col in data_frame.columns if f'value_{year}' in col]
#     data_frame = data_frame[(data_frame[relevant_cols[0]] == 13) & (data_frame['sctg2'] == commodity_code)]
#     data_frame = data_frame[relevant_cols + year_specific_col]
#     data_frame = data_frame.dropna()
#     data_frame = data_frame.drop(columns=[relevant_cols[0],'sctg2'])
#     data_frame = data_frame.sum(axis=0)
#     return data_frame

In [33]:
# df_2012 = filter_georgia_value(df_2012, 2012, 5)
# df_2013 = filter_georgia_value(df_2013, 2013, 5)
# df_2016 = filter_georgia_value(df_2016, 2016, 5)
# df_2017faf4 = filter_georgia_value(df_2017faf4, 2017, 5)
# df_2017faf5 = filter_georgia_value(df_2017faf5, 2017, 5)
# df_2018faf4 = filter_georgia_value(df_2018faf4, 2018, 5)
# df_2018faf5 = filter_georgia_value(df_2018faf5, 2018, 5)

In [51]:
# combined_sums = pd.DataFrame({
#     '2012': df_2012.iloc[0],
#     '2013': df_2013.iloc[0],
#     '2016': df_2016.iloc[0],
#     '2017': df_2017faf4.iloc[0],
#     '2017faf5': df_2017faf5.iloc[0],
#     '2018faf4': df_2018faf4.iloc[0],
#     '2018faf5': df_2018faf5.iloc[0]
# })

In [56]:
# plot_data = {
#     2012: {'FAF4': df_2012.iloc[0], 'FAF5': np.nan}, 
#     2013: {'FAF4': df_2013.iloc[0], 'FAF5': np.nan}, 
#     2016: {'FAF4': df_2016.iloc[0], 'FAF5': np.nan}, 
#     2017: {'FAF4': df_2017faf4.iloc[0], 'FAF5': df_2017faf5.iloc[0]}, 
#     2018: {'FAF4': df_2018faf4.iloc[0], 'FAF5': df_2018faf5.iloc[0]}, 
# }
# 
# plot_df = pd.DataFrame.from_dict(plot_data, orient='index')
# plot_df.index.name = 'Year' 
# 
# plot_df.index = plot_df.index.astype(int)
# 
# plt.figure()
# plot_df.plot(ax=plt.gca(), marker='o', linewidth=2) 
# 
# plt.title('Value Over Years (FAF4 vs FAF5 Methodologies)')
# plt.xlabel('Year')
# plt.ylabel('Total Value for Commodity Group 5 (Meat/Seafood in Georgia)')
# plt.grid(True)
# plt.legend(title='FAF Version')
# plt.xticks(plot_df.index) 
# plt.tight_layout()
# plt.show()

In [62]:
# def commodity_based_value_trend(commodity_code):
#     df_2012 = pd.read_csv('freight_data/FAF5.6.1_Reprocessed_1997-2012_State.csv')
#     df_2012faf4 = pd.read_csv('freight_data/FAF4.5.1_State.csv')
#     df_2013 = pd.read_csv('freight_data/FAF4.5.1_State_2013.csv')
#     df_2014 = pd.read_csv('freight_data/FAF4.5.1_State_2014.csv')
#     df_2015 = pd.read_csv('freight_data/FAF4.5.1_State_2015.csv')
#     df_2016 = pd.read_csv('freight_data/FAF4.5.1_State_2016.csv')
#     df_2017faf4 = pd.read_csv('freight_data/FAF4.5.1_State_2017.csv')
#     df_2017faf5 = pd.read_csv('freight_data/FAF5.6.1_State.csv')
#     df_2018faf4 = pd.read_csv('freight_data/FAF4.5.1_State_2018.csv')
#     df_2018faf5 = pd.read_csv('freight_data/FAF5.6.1_State_2018-2023.csv')
#     
#     df_2012 = filter_georgia_value(df_2012, 2012, commodity_code)
#     df_2012faf4 = filter_georgia_value(df_2012faf4, 2012, commodity_code)
#     df_2013 = filter_georgia_value(df_2013, 2013, commodity_code)
#     df_2014 = filter_georgia_value(df_2014, 2014, commodity_code)
#     df_2015 = filter_georgia_value(df_2015, 2015, commodity_code)
#     df_2016 = filter_georgia_value(df_2016, 2016, commodity_code)
#     df_2017faf4 = filter_georgia_value(df_2017faf4, 2017, commodity_code)
#     df_2017faf5 = filter_georgia_value(df_2017faf5, 2017, commodity_code)
#     df_2018faf4 = filter_georgia_value(df_2018faf4, 2018, commodity_code)
#     df_2018faf5 = filter_georgia_value(df_2018faf5, 2018, commodity_code)
# 
#     plot_data_dict = {
#         2012: {'FAF4': df_2012faf4.iloc[0], 'FAF5': df_2012.iloc[0]}, 
#         2013: {'FAF4': df_2013.iloc[0], 'FAF5': np.nan}, 
#         2014: {'FAF4': df_2014.iloc[0], 'FAF5': np.nan},
#         2015: {'FAF4': df_2015.iloc[0], 'FAF5': np.nan},
#         2016: {'FAF4': df_2016.iloc[0], 'FAF5': np.nan}, 
#         2017: {'FAF4': df_2017faf4.iloc[0], 'FAF5': df_2017faf5.iloc[0]}, 
#         2018: {'FAF4': df_2018faf4.iloc[0], 'FAF5': df_2018faf5.iloc[0]}, 
# }
#     return plot_data_dict
#     
# def plot_from_data_dict(plot_data, commodity_code):
#     plot_df = pd.DataFrame.from_dict(plot_data, orient='index')
#     plot_df.index.name = 'Year' 
#     
#     plot_df.index = plot_df.index.astype(int)
#     
#     plt.figure()
#     plot_df.plot(ax=plt.gca(), marker='o', linewidth=2) 
#     
#     commodity_mapping = {5: 'Meat/seafood', 8: 'Alcoholic beverages', 
#                          9: 'Tobacco prods.', 21: 'Pharmaceuticals'}
#     
#     plt.title(f'Value/Years: {commodity_mapping[commodity_code]}\n in Georgia (FAF4 vs FAF5 Methodologies)')
#     plt.xlabel('Year')
#     plt.ylabel(f'Shipment Value Sum\n ({commodity_mapping[commodity_code]} in Georgia)')
#     plt.grid(True)
#     plt.legend(title='FAF Version')
#     plt.xticks(plot_df.index) 
#     plt.tight_layout()
#     plt.show()

In [2]:
plot_data_trend = commodity_based_value_trend(commodity_code=8)
plot_from_data_dict(plot_data_trend, commodity_code=8)

In [3]:
plot_data_trend = commodity_based_value_trend(commodity_code=9)
plot_from_data_dict(plot_data_trend, commodity_code=9)

In [4]:
plot_data_trend = commodity_based_value_trend(commodity_code=21)
plot_from_data_dict(plot_data_trend, commodity_code=21)

## Day 11

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
df = pd.read_csv('../freight_data/processed/Georga_AIS_2012-2023_minus_inflation.csv', index_col=0)

georgia_with_inflation = pd.read_csv('../freight_data/processed/Georgia_Annual_Inbound_Shipments_2012-2023.csv', index_col=0)

In [4]:
df

In [5]:
georgia_with_inflation

In [6]:
combined_df = pd.concat([df[['value_5', 'tons_5']], georgia_with_inflation[['value_5', 'tons_5']]], axis=1)

combined_df.plot(figsize=(10, 6), marker='o')
plt.xlabel("Year")
plt.ylabel("Value / Tons")
plt.title("Shipment Trends for A and B")
plt.grid(True)
plt.tight_layout()
plt.show()

In [9]:
df_renamed = df[['value_5', 'tons_5']].rename(columns={
    'value_5': 'Value (Original)',
    'tons_5': 'Tons (Original)'
})
georgia_renamed = georgia_with_inflation[['value_5', 'tons_5']].rename(columns={
    'value_5': 'Value (Inflation-Adjusted)',
    'tons_5': 'Tons (Inflation-Adjusted)'
})
combined_df = pd.concat([df_renamed, georgia_renamed], axis=1)

In [12]:
import plotly.graph_objects as go

fig = go.Figure()

for column in combined_df.columns:
    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=combined_df[column],
        mode='lines+markers',
        name=column
    ))

fig.update_layout(
    title='Shipment Trends (Original vs Inflation-Adjusted)',
    xaxis_title='Year',
    yaxis_title='Value (Million Dollars) / Tons (Million Tons)',
    legend_title='Metric',
    template='plotly_white',
    # width=900,
    # height=500
)

fig.show()
